In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from mediawiki import MediaWiki
from bs4 import BeautifulSoup
import string
from nltk import tokenize
from nltk.tag import StanfordNERTagger
import string
from nltk.tokenize import word_tokenize
import unicodedata
import timeit
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
# Using the Stanfors' default tagger
st = StanfordNERTagger('/Users/venkatakrishnamohansunkara/Desktop/DM/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz',
  '/Users/venkatakrishnamohansunkara/Desktop/DM/stanford-ner-2018-02-27/stanford-ner.jar',
   encoding='utf-8') 

In [3]:
# Lists containing common prefixes and suffixes.
prefix = ['at','by','near','in','from', 'on','to']
# https://www.irfca.org/docs/place-names.html
suffixes = ['nagar','colony','street','road','hill','river','temple']
subs = ['pur','puram','ghar','pura','ganj','abad','halli','keri']

In [4]:
preps_score={'"': 0.99,
 'down': 0.99,
 'across': 0.25,
 'near': 0.67,
 'inside': 0.23,
 'amid': 0.23,
 'in': 0.22,
 'from': 0.205,
 'off': 0.2,
 'at': 0.19,
 'like': 0.18,
 'outside': 0.12,
 'because': 0.1,
 'of': 0.09,
 'about': 0.08,
 'between': 0.075,
 'by': 0.07,
 'among': 0.07,
 'with': 0.07,
 'against': 0.07,
 'into': 0.06,
 'that': 0.05,
 'through': 0.045,
 'during':0.045,
 'after': 0.045,
 'on': 0.03,
 'while':0.03,
 'for': 0.02,
 'as': 0.015}

In [5]:
vocab = ["strike",
"unrest",
"masses",
"protest",
"demonstration",
"worker",
"union",
"company",
"caste",
"religious",
"ethnic",
"reformed",
"rebellion",
"defense",
"violence",
"war",
"armed",
"fight",
"Right",
"free",
"freedom",
"liberty",
"justice",
"Fair",
"unfair",
"unequal",
"terror",
"extreme",
"Bomb",
"IED",
"weapon",
"gun",
"wmd",
"threat",
"suicide",
"murder",
"Kill",
"death",
"explosive",
"military",
"police",
"elite",
"government",
"oppresive",
"power",
"regime",
"fraud",
"corruption",
"coup",
"safety",
"secure",
"insecure",
"protect",
"enemy",
"resist",
"hostage",
"truce",
"fire",
"greed",
"panic",
"inflation",
"Price",
"Food",
"Water"]

In [6]:
# Read the data
#news_data = pd.read_csv('/Users/venkatakrishnamohansunkara/Desktop/DM/GDELT_2017/large_dataset/unrest_2017_ie_toi.csv', sep=',')
df = pd.read_csv('../document similarity/document_similarity_hatt.csv')

In [3]:
all_locs = pd.read_csv('all_locations.csv')

In [5]:
all_locs.shape

(1315065, 6)

In [8]:
# Remove non english characters, everything in ()
# Sort the dataframe
for i,row in all_locs.iterrows():
    aname = row['Area Name']
    aname = re.sub("[\(\[].*?[\)\]]", "", str(aname))
    all_locs.set_value(i,'Area Name',aname.strip().lower())

In [9]:
test_data = df.iloc[1].content

In [10]:
# Function for removing unnecessary characters.
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = unicodedata.normalize("NFKD", string)
    string = re.sub(r"\\", " ", string)
    string = re.sub(r"\'", " ", string)
    string = re.sub(r"\"", " ", string)
    string = re.sub(r"\n", " ", string)
    string = re.sub(r"=", " ", string)
    string = re.sub(r"-", " ", string)
    string = re.sub(r"/", " ", string)
    return string.strip()

In [11]:
text_sent = nltk.sent_tokenize(test_data)
text_tags = nltk.word_tokenize(text_sent[0])
text_tagged = nltk.pos_tag(text_tags)

In [12]:
text_tagged

[('Anna', 'NNP'),
 ('Hazare', 'NNP'),
 ('on', 'IN'),
 ('fast-unto-death', 'JJ'),
 ('demanding', 'VBG'),
 ('Jan', 'NNP'),
 ('Lokpal', 'NNP'),
 ('Bill', 'NNP'),
 ('Veteran', 'NNP'),
 ('Gandhian', 'NNP'),
 (',', ','),
 ('Mr', 'NNP'),
 ('Anna', 'NNP'),
 ('Hazare', 'NNP'),
 (',', ','),
 ('began', 'VBD'),
 ('a', 'DT'),
 ('fast-unto-death', 'NN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('Capital', 'NN'),
 ('on', 'IN'),
 ('Tuesday', 'NNP'),
 ('demanding', 'VBG'),
 ('enactment', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('comprehensive', 'JJ'),
 ('Jan', 'NNP'),
 ('Lokpal', 'NNP'),
 ('Bill', 'NNP'),
 ('to', 'TO'),
 ('check', 'VB'),
 ('corruption', 'NN'),
 ('.', '.')]

In [13]:
def get_locations(test_data):
    # Convert the entire document data to sentences.
    text_sent = nltk.sent_tokenize(test_data)
    stanford = []
    noun_phrases = []
    #new_model = []
    locations = []
    # other are prefixes
    other = []
    suffix_locations = []
    sub_locations = []
    all_words = []
    # Iterate over all the sentences.
    for sent in text_sent:
        # Convert the sentence to words
        text_tags = nltk.word_tokenize(sent)
        all_words+=[text_tags]
        # Get the NER for the words.
        original_tags = st.tag(text_tags)
        l = len(original_tags)
        i=0;
        # Iterate over the tagged words.
        while i<l:
            #print(i)
            e,t = original_tags[i];
            # If it's a location, then check the next 3 words.
            if t == 'LOCATION':
                j = 1;
                s = e; 
                # Verify the tags for the next 3 words.
                while i+j<len(original_tags):
                    # If the next words are also locations, then concatenate them to make a large string.
                    if original_tags[i+j][1] == 'LOCATION':
                        s = s+" "+original_tags[i+j][0];
                        j+=1;
                    else:
                        break;
                i = i+j;
                # Save the locations to a locations list
                locations+=[s];
            else:
                i=i+1;
        # Get the POS tag for the words.
        text_tagged = nltk.pos_tag(text_tags)
        l = len(text_tagged)
        i=0;
        # iterate over the tagged words.
        while i<l:
            #print(i)
            e,t = text_tagged[i];
            # If the current word is a Noun phrase.
            if t == 'NNP':
                j = 1;
                s = e;
                # verify the tags for the next 3 words.
                while i+j<len(text_tagged):
                    # If the next words are also nouns, then concatenate them to make a large string.
                    if text_tagged[i+j][1] in ['NNP','NN','NNS','NNPS']:
                        s = s+" "+text_tagged[i+j][0];
                        j+=1;
                    else:
                        break;
                i = i+j;
                # Save the noun phrases to a noun_phrases list
                noun_phrases+=[s];
            else:
                i=i+1;
        # Convert the words to lower case so as to compare with the prefixes.
        text_tags = [t.lower() for t in text_tags]
        # Go through all the prefixes.
        for k in prefix:
            # If the prefix is not present in the sentence then an error occurs. So, we use a try, catch block
            try:
                # Get the position the prefix word in a sentence.
                index = text_tags.index(k)
                # If the word after the prefix word is a noun, then consider it.
                if text_tagged[index+1][1] == 'NNP':
                    j = 1;
                    s = text_tagged[index+1][0]; 
                    # Verify if it is a phrase by considering the next 3 words.
                    while index+j<len(text_tagged):
                        if text_tagged[index+j+1][1] in ['NNP','NN','NNS','NNPS']:
                            s = s+" "+text_tagged[index+j+1][0];
                            j+=1;
                        else:
                            break;
                    # Save the words after the prefix words in to other list.
                    other = other + [s]
            except:
                continue
        # Iterate through the list of suffixes.
        for s in suffixes:
            try:
                # get the position of the suffix word in the sentence
                index = text_tags.index(s)
                # If the word before a suffix word is a noun phrase, then consider it.
                if text_tagged[index-1][1] == 'NNP':
                    j = 1;
                    stri = text_tagged[index-1][0]; 
                    # verify if the preceeding 3 words are nouns.
                    while index+j<len(text_tagged):
                        if text_tagged[index-j-1][1] in ['NNP','NN','NNS','NNPS']:
                            stri = stri+" "+text_tagged[index-j-1][0];
                            j+=1;
                        else:
                            break;
                    # Reverse the preceeding words list
                    stri = ' '.join(stri.split(" ")[-1::-1]) 
                    # If the string is not already present in other lists, then add it to the suffix_locations list.
                    if stri not in locations or stri not in noun_phrases or stri not in other:
                        stri = stri+" "+ s;
                        if stri not in locations or stri not in noun_phrases or stri not in other:
                            suffix_locations = suffix_locations + [stri]
            except:
                continue
        for sub in subs:
            # get the words which contain any of the sub_string from the subs list.
            su = [te for te in text_tags if te.endswith(sub)]
            try:
                for sub1 in su:
                    # Get the position of the word from the sentence.
                    index = text_tags.index(sub1)
                    # If it is a Noun, then consider it.
                    if text_tagged[index][1] in ['NNP','NN','NNS','NNPS']:
                        sub_locations = sub_locations + [text_tagged[index][0]]
            except:
                continue
    all_words = [x for sublist in all_words for x in sublist]
    all_locations = locations+other+suffix_locations+sub_locations
    all_locations_nodups = []
    for i in all_locations:
        if i.lower() not in [j.lower() for j in all_locations_nodups]:
            all_locations_nodups.append(i);
    #Locations: Stanford
    #Other: Prepositions
    #suffix_locations: suffixes
    #sub_locations: sub_strings
    return locations,other,suffix_locations,sub_locations,all_locations_nodups, all_words, text_sent

In [14]:
def get_location_sources(locations,other,suffix_locations,sub_locations,all_locations_nodups):
    location_sources = {}
    for loc in all_locations_nodups:
        a = []
        if loc in locations:
            a.append('stanford')
        if loc in other:
            a.append('preps')
        if loc in suffix_locations:
            a.append('suffixes')
        if loc in sub_locations:
            a.append('sub_strings')
        if loc not in location_sources:
            location_sources[loc] = a
    return location_sources

In [15]:
def linear_decay(i,n):
    return 1-i/n
def exp_decay(i,n):
    return np.exp(-(i/n))

In [16]:
#get unrest and position occurences of all locations.
def get_unrest_words_pos(all_words,locations_list,decay):
    pos_threshold = len(all_words)
    all_locations = {k:[] for k in locations_list}
    nearest_locations = {k:[] for k in locations_list}
    pos_occurences = {k:[] for k in locations_list}
    pos_scores = {k:-1 for k in pos_occurences.keys()}
    unrest_scores = {k:-1 for k in all_locations.keys()}
    for location in list(all_locations.keys()):
        sub_locations = location.split(' ')
        if len(sub_locations) > 1:
            last_subloc = sub_locations[-1]
            first_subloc = sub_locations[0]
            first_indices = [i for i, w in enumerate(all_words) if w == first_subloc]
            last_indices = [i for i, w in enumerate(all_words) if w == last_subloc]
            diff = [a_i - b_i for a_i, b_i in zip(last_indices, first_indices)]
            first_indices = first_indices[diff == len(sub_locations)]
            if not isinstance(first_indices,list):
                pos_occurences[location].append(first_indices)
            else:
                pos_occurences[location] = first_indices
            indices = pos_occurences[location]
        else:
            first_indices = [i for i, w in enumerate(all_words) if w == sub_locations[0]]
            if not isinstance(first_indices,list):
                pos_occurences[location].append(first_indices)
            else:
                pos_occurences[location] = first_indices
            indices = pos_occurences[location]
        if len(indices)>0:
            no_of_indices = len(indices)
            i=0
            b=[]
            nearest_locs = []
            while i< no_of_indices:
                j=0
                a = []
                l = []
                while j<pos_threshold and indices[i]+j<len(all_words):
                    if all_words[indices[i]+j].lower() in vocab:
                        s = [j,indices[i],'r',all_words[indices[i]+j].lower()]
                        a.append(s)
                    if all_words[indices[i]+j] in locations_list:
                        lo = [j,indices[i],'r',all_words[indices[i]+j].lower()]
                        l.append(lo)
                    j=j+1
                j=0
                while j<pos_threshold and indices[i]-j>=0:
                    if all_words[indices[i]-j].lower() in vocab:
                        s = [j,indices[i],'l',all_words[indices[i]-j].lower()]
                        a.append(s)
                    if all_words[indices[i]-j] in locations_list:
                        lo = [j,indices[i],'l',all_words[indices[i]-j].lower()]
                        l.append(lo)
                    j=j+1
                i=i+1
                b = b+a
                nearest_locs = nearest_locs+l
            all_locations[location] = b
            nearest_locations[location] = nearest_locs
    for i in pos_scores.keys():
        a = []
        for j in pos_occurences[i]:
            if decay == 'linear':
                a.append(linear_decay(j,len(all_words)))
            else:
                a.append(exp_decay(j,len(all_words)))
        pos_scores[i] = a
    for i in unrest_scores.keys():
        a = []
        for j in all_locations[i]:
            if decay == 'linear':
                a.append(linear_decay(j[0],len(all_words)))
            else:
                a.append(exp_decay(j[0],len(all_words)))
        unrest_scores[i] = a
    #all_locations: all unrest keywords from all mentions of a location
    #nearest location: all other locations from all mentions of a location
    #pos_occurences: The positions of locations
    #pos_scores, unrest_scores: Scores for a location mentions based on their positon and the unrest keywords
    return all_locations,nearest_locations,pos_occurences,pos_scores,unrest_scores

In [17]:
def get_unrest_words_sent(sentence_no,text_sent,vocab,decay):
    unrest_sent = {k:[] for k in sentence_no.keys()}
    other_locs_sent = {k:[] for k in sentence_no.keys()}
    lower_keys = [k.lower() for k in sentence_no.keys()]
    for i in sentence_no.keys():
        sents = sentence_no[i]
        a=[]
        b=[]
        for j in sents:
            words = nltk.word_tokenize(text_sent[j].lower())
            unrest_words_in_sent = list(set(words) & set(vocab))
            other_locs_in_sent = list(set(words)&set(lower_keys))
            if len(unrest_words_in_sent) != 0:
                s = [i,j,unrest_words_in_sent]
                a.append(s)
            if len(other_locs_in_sent) != 0:
                s1 = [i,j,other_locs_in_sent]
                b.append(s1)
        unrest_sent[i] = a
        other_locs_sent = b
    #unrest_sent: all unrest keywords with in the same sentence as a location mention
    #other_locs_sent: all other location mentions in the same sentence as a location mention.
    return unrest_sent,other_locs_sent

In [37]:
def get_unrest_density_score(unrest_locs,unrest_sent,thresh=None):
    unrest_scores = {k:-1 for k in unrest_locs.keys()}
    for l in unrest_sent.keys():
        score =0;
        if len(unrest_sent[l]) >0:
            sent_unrest = unrest_sent[l][0][2]
            locs_unrest = unrest_locs[l]
            for s in sent_unrest:
                for l2 in locs_unrest:
                    if s == l2[3]:
                        score = score+(1/l2[0]);
                    else:
                        if thresh!=None:
                            if l2[0]<thresh:
                                score = score+np.exp(-l2[0]);  
                        else:
                            score = score+np.exp(-l2[0]); 
        else:
            for l2 in unrest_locs[l]:
                if thresh!=None:
                    if l2[0]<thresh:
                        score = score+np.exp(-l2[0]);  
                else:
                    score = score+np.exp(-l2[0]); 
        unrest_scores[l] = score/len(unrest_locs[l]);
    return unrest_scores

In [19]:
def get_frequecies(pos_occurences,all_locations):
    locations_freq = {i:0 for i in all_locations}
    for i in pos_occurences.keys():
        locations_freq[i] = len(pos_occurences[i])
    #locatoins_Freq: Number of occurences of each location in a document.
    return locations_freq

In [20]:
def get_sent_nos(locations_list,text_sent,decay):
    sentence_no = {k:[] for k in locations_list}
    sentence_scores = {k:-1 for k in locations_list}
    for loc in locations_list:
        sub_locations = loc.split(' ')
        sent_no = 0
        for sent in text_sent:
            text_tags = nltk.word_tokenize(sent)
            i=0
            for s in sub_locations:
                if s in text_tags:
                    i+=1
            if i == len(sub_locations):
                sentence_no[loc] = sentence_no[loc]+[sent_no]
            sent_no+=1
    for loc in locations_list:
        a=[]
        for i in sentence_no[loc]:
            if decay == 'linear':
                a.append(linear_decay(i,len(text_sent)))
            else:
                a.append(exp_decay(i,len(text_sent)))
        sentence_scores[loc] = a
    # sentence_no: The sentence numbers where a location is mentioned in an article.
    return sentence_no,sentence_scores

In [21]:
def get_gazette_score(all_locations,all_locs):
    gazette_score = {k:-1 for k in all_locations}
    for name in all_locations:
        def get_ratio(row):
            aname = row['Area Name']
            return fuzz.ratio(aname,name.lower())
        # Extract the closest match > 98%.
        # If more than one extract all 
        found_locs = all_locs[all_locs.apply(get_ratio, axis=1) > 85]
        if found_locs.shape[0]>0:
            gazette_score[name] = 1
        else:
            gazette_score[name] = 0
    # gazette_Score: score which verifies whether a location is present in the list of gazette locations.
    return gazette_score

In [22]:
def get_title_score(all_locations, title):
    title_score = {k:-1 for k in all_locations}
    for i in all_locations:
        if i in title:
            title_score[i]=1
        else:
            title_score[i]=0
    #title_score: score which verifies whether a location is mentioned in the title or not.
    return title_score

In [23]:
def get_preps_score(all_locations,all_words,preps_score,pos_scores):
    preps_scores = {k:[] for k in all_locations}
    for loc in all_locations:
        sub_locations = loc.split(' ')
        indices = pos_scores[loc]
        s = []
        for i in indices:
            try:
                if i-1>0 and all_words[i-1] in preps_score.keys():
                    s.append(preps_score[all_words[i-1]])
                else:
                    s.append(0);   
            except:
                    continue
        preps_scores[loc] = s
    # preps_scores: Scores to each location mention in an article based on the prepositions preceding them.
    return preps_scores

In [24]:
def get_final_scores(all_locations,unrest_score,preps_score,title_score,sent_score,position_score,gazette_score,c=6):
    final_scores = {k:-1 for k in all_locations}
    if c == 6:
        for i in all_locations:
            score = unrest_score[i]+max(pos_scores[i])+max(preps_score[i])+max(sent_score[i])+title_score[i]+gazette_score[i]
            final_scores[i] = score/6;
    else:
        for i in all_locations:
            score = max(pos_scores[i])+max(preps_score[i])+max(sent_score[i])+title_score[i]+gazette_score[i]
            final_scores[i] = score/5;
    # final_scores: A combination of all the above scores for a location.
    return final_scores

In [25]:
test_data = clean_str(test_data)

In [26]:
locations,other,suffix_locations,sub_locations,all_locations,all_words,text_sents = get_locations(test_data)

In [27]:
location_sources = get_location_sources(locations,other,suffix_locations,sub_locations,all_locations)

In [28]:
unrest_locs,nearest_locations,pos_locs,pos_scores,unrest_scores = get_unrest_words_pos(all_words,all_locations,'linear')

In [29]:
freqs = get_frequecies(pos_locs,all_locations)

In [30]:
sent_nos,sent_scores = get_sent_nos(all_locations,text_sents,'linear')

In [31]:
unrest_sent,other_locs_sent = get_unrest_words_sent(sent_nos,text_sents,vocab,'linear')

In [32]:
title_scores = get_title_score(all_locations,text_sents[0])

In [33]:
preps_scores = get_preps_score(all_locations,all_words,preps_score,pos_locs)

In [34]:
gazette_scores = get_gazette_score(all_locations,all_locs)

In [35]:
unrest_scores = get_unrest_density_score(unrest_locs,unrest_sent)

In [36]:
final_scores = get_final_scores(all_locations,unrest_scores,preps_scores,title_scores,sent_scores,pos_scores,gazette_scores)

In [77]:
import math
unrest_scores = {k:-1 for k in unrest_locs.keys()}
for l in unrest_locs.keys():
    a = unrest_locs[l]
    mini = math.inf;
    for x in a:
        if x[0]<mini:
            mini = x[0]
    unrest_scores[l] = 1 - mini/len(all_words)

In [ ]:
unrest_sent

In [86]:
sent_scores = {'Ahmednagar': [0.528],
 'Azad Maidan': [0.617],
 'Central Mumbai': [0.617],
 'Jantar Mantar': [0.526],
 'Maharashtra': [0.528],
 'Mahatma Gandhi': [0.598],
 'Monday night': [0.627],
 'Mumbai': [0.617],
 'Rajghat': [0.598, 0.669],
 'Ralegan Siddhi': [0.528],
 'Shivaji Park': [0.617]}

In [88]:
final_scores

{'Ahmednagar': 0.4789380234505862,
 'Azad Maidan': 0.3099606365159129,
 'Central Mumbai': 0.2948601340033501,
 'Jantar Mantar': 0.37728978224455617,
 'Maharashtra': 0.44101507537688445,
 'Mahatma Gandhi': 0.3982428810720268,
 'Monday night': 0.34570603015075374,
 'Mumbai': 0.4791147403685092,
 'Rajghat': 0.6058969849246231,
 'Ralegan Siddhi': 0.44478391959798996,
 'Shivaji Park': 0.46153936348408714}

In [75]:
count={}
for u in unrest_sent.keys():
    c = unrest_sent[u]
    if len(c) == 0:
        try:
           count[0]+=1;
        except:
            count[0] = 1;
    for s in c:
        l = s[2];
        try:
           count[len(l)]+=1;
        except:
            count[len(l)] = 1;

In [76]:
count

{0: 4, 1: 7, 2: 1}

In [88]:
x = pd.DataFrame(list(count.items()), columns=['count', 'countValue'])
x.append(x).groupby('count').sum()

,countValue
count,
0,8
1,14
2,2


In [71]:
final_scores

{'Ahmednagar': 0.09877792317907393,
 'Azad Maidan': 0.07744945411697447,
 'Central Mumbai': 0.05866915620718632,
 'Jantar Mantar': 0.2550914830563072,
 'Maharashtra': 0.06205114646716531,
 'Mahatma Gandhi': 0.295387192372117,
 'Monday night': 0.2022290941888932,
 'Mumbai': 0.08303736749016066,
 'Rajghat': 0.3592430707653918,
 'Ralegan Siddhi': 0.06245846483239976,
 'Shivaji Park': 0.05795877303251321,
 'tricolour': 0.2304851178971782}

In [46]:
location_sources

{'Ahmednagar': ['stanford'],
 'Azad Maidan': ['stanford', 'preps'],
 'Central Mumbai': ['stanford'],
 'Jantar Mantar': ['stanford', 'preps'],
 'Maharashtra': ['stanford'],
 'Mahatma Gandhi': ['preps'],
 'Monday night': ['preps'],
 'Mumbai': ['stanford', 'preps'],
 'Rajghat': ['preps'],
 'Ralegan Siddhi': ['stanford'],
 'Shivaji Park': ['stanford', 'preps'],
 'tricolour': ['sub_strings']}

In [71]:
test_data

'Anna Hazare on fast unto death demanding Jan Lokpal Bill  Veteran Gandhian, Mr Anna Hazare, began a fast unto death in the Capital on Tuesday demanding enactment of a comprehensive Jan Lokpal Bill to check corruption. Ignoring an appeal by the Prime Minister, the 72 year old social activist began his day by paying tribute to Mahatma Gandhi at Rajghat. He was joined by activists such as Swami Agnivesh, former IPS officer and activist Ms Kiran Bedi and Magsaysay award winner Mr Sandeep Pandey. “I will observe fast unto death till the Government agrees to form a joint committee comprising 50 per cent officials and the remaining citizens and intellectuals to draft the Jan Lokpal Bill,” Mr Hazare said at Rajghat. On the way to the fast site at Jantar Mantar, Mr Hazare s open jeep was greeted by tricolour waving supporters, including school children. On Monday night, the Prime Minister s Office had said that the Prime Minister had enormous respect for Mr Hazare and his mission. “The Prime M

In [26]:
a = []

In [29]:
a.append(2)

In [30]:
a

[2, 2]

In [15]:
'Assam' in list(all_locs['Area Name'])

True